# Capítulo 6, ejercicio propuesto 8. Deducción del gradiente y del laplaciano en coordenadas esféricas

$$
\newcommand{\ve}[1]{{\boldsymbol{#1}}}
\newcommand{\ma}[1]{{\boldsymbol{#1}}}
\newcommand{\hati}{\ve{\hat i}}
\newcommand{\hatj}{\ve{\hat j}}
\newcommand{\hatk}{\ve{\hat k}}
\newcommand{\hatr}{\ve{\hat r}}
\newcommand{\hatt}{\ve{\hat \theta}}
\newcommand{\hatp}{\ve{\hat \varphi}}
$$

In [3]:
from sympy import init_printing, latex, symbols, sin, cos, diff, simplify, Function, Matrix, trigsimp,  expand

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [4]:
r, t, p = symbols('r, theta, varphi')
x = r*sin(t)*cos(p)
y = r*sin(t)*sin(p)
z = r*cos(t)

# se define la transpuesta de la matriz Jacobiana
dxyz_drtp_T = Matrix([[diff(x,r),  diff(y,r),  diff(z,r)],
                      [diff(x,t),  diff(y,t),  diff(z,t)],
                      [diff(x,p),  diff(y,p),  diff(z,p)]])

imprimir(r"\frac{\partial(x,y,z)}{\partial(r,\theta,\varphi)}^T = ", dxyz_drtp_T)

<IPython.core.display.Math object>

In [5]:
# se invierte la transpuesta de la matriz Jacobiana
drtp_dxyz_T = simplify(dxyz_drtp_T**-1)

imprimir(r"\frac{\partial(r,\theta,\varphi)}{\partial(x,y,z)}^T = ", drtp_dxyz_T)

<IPython.core.display.Math object>

In [6]:
# se especifican las derivadas a partir de la matriz Jacobiana calculada anteriormente
dr_dx, dt_dx, dp_dx = drtp_dxyz_T[0,:]
dr_dy, dt_dy, dp_dy = drtp_dxyz_T[1,:]
dr_dz, dt_dz, dp_dz = drtp_dxyz_T[2,:]

Se especifican "manualmente" las primeras derivadas utilizando la regla de la cadena para derivar una función $f(r,\theta,\phi)$ teniendo en cuenta que $r$, $\theta$ y $\phi$ son funciones de $x$, $y$ y $z$:
\begin{align*}
\frac{\partial f}{\partial x} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial x} + \frac{\partial f}{\partial \theta} \frac{\partial \theta}{\partial x} + \frac{\partial f}{\partial \varphi} \frac{\partial \varphi}{\partial x}\\
\frac{\partial f}{\partial y} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial y} + \frac{\partial f}{\partial \theta} \frac{\partial \theta}{\partial y} + \frac{\partial f}{\partial \varphi} \frac{\partial \varphi}{\partial y} \\
\frac{\partial f}{\partial z} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial z} + \frac{\partial f}{\partial \theta} \frac{\partial \theta}{\partial z} + \frac{\partial f}{\partial \varphi} \frac{\partial \varphi}{\partial z}
\end{align*}    

In [8]:
d_dx = lambda f: diff(f,r)*dr_dx + diff(f,t)*dt_dx + diff(f,p)*dp_dx
d_dy = lambda f: diff(f,r)*dr_dy + diff(f,t)*dt_dy + diff(f,p)*dp_dy
d_dz = lambda f: diff(f,r)*dr_dz + diff(f,t)*dt_dz + diff(f,p)*dp_dz

Se definen las segundas derivadas:

In [10]:
d2_dx2 = lambda f: d_dx(d_dx(f))
d2_dy2 = lambda f: d_dy(d_dy(f))
d2_dz2 = lambda f: d_dz(d_dz(f))

Se definen los vectores unitarios:
\begin{align*}
    \ve{\hat{r}}       &= \sin\theta \cos\varphi \hati + \sin\theta \sin\varphi \hatj + \cos\theta \hatk \\
    \ve{\hat{\theta}}  &= \cos\theta \cos\varphi \hati + \cos\theta \sin\varphi \hatj - \sin\theta \hatk \\
    \ve{\hat{\varphi}} &= -\sin\varphi \hati + \cos \varphi \hatj.
\end{align*}

In [12]:
rg = Matrix([ sin(t)*cos(p), sin(t)*sin(p),  cos(t) ])
tg = Matrix([ cos(t)*cos(p), cos(t)*sin(p), -sin(t) ])
pg = Matrix([       -sin(p),        cos(p),       0 ])

Y luego se define la matriz de transformación como $\ma{T} = \left[\ve{\hat{r}},\ \ve{\hat{\theta}},\ \ve{\hat{\varphi}}  \right]$

In [14]:
T = Matrix.hstack(rg, tg, pg)

imprimir(r"\boldsymbol{T} = ", T)

<IPython.core.display.Math object>

In [15]:
# se especifica que f es función de r, t y p
f = Function('f')(r, t, p)

## Caĺculo del laplaciano en coordenadas esféricas

Se especifica el laplaciano en coordenadas rectangulares:

In [18]:
lapl1 = simplify(d2_dx2(f) + d2_dy2(f) + d2_dz2(f))

Se imprime el laplaciano en coordenadas esféricas:

In [20]:
imprimir(r"\nabla^2 \phi(r, \theta) = ", lapl1)

<IPython.core.display.Math object>

y se compara contra la fórmula que está en el libro:
$$
\nabla^2 f = \frac{1}{r^2}\frac{\partial}{\partial r}\left(r^2 \frac{\partial f}{\partial r}\right) + \frac{1}{r^2\sin\theta}\frac{\partial}{\partial \theta}\left(\sin\theta \frac{\partial f}{\partial \theta}\right) + \frac{1}{r^2\sin^2\theta}\frac{\partial^2 f}{\partial \varphi^2}
$$

In [22]:
lapl2 = (1/(r**2))*diff(r**2 * diff(f,r), r) + (1/(r**2 * sin(t)))*diff(sin(t) * diff(f,t), t) + (1/(r**2 * sin(t)**2))*diff(f,p,2)

Si `lapl1 - lapl2 == 0` es porque ambas expresiones son matemáticamente equivalentes:

In [24]:
simplify(lapl1 - lapl2)

## Cálculo del gradiente en coordenadas esféricas:

Se especifica el gradiente en coordenadas rectangulares, el cual está definido con respecto a la base $\left\{\hati,\ \hatj, \hatk \right\}$:

In [27]:
grad_f_ijk = Matrix([ d_dx(f),
                      d_dy(f),
                      d_dz(f)  ])

Se convierte el gradiente de la base $\left\{\hati,\ \hatj, \hatk \right\}$ a la base $\left\{\hatr,\ \hatt, \hatp \right\}$:

In [29]:
grad_f_rtp = trigsimp(T.T * grad_f_ijk)

Se imprime el gradiente en coordenadas esféricas:

In [31]:
imprimir(r"\boldsymbol{\nabla} f(r, \theta, \varphi) = ", grad_f_rtp)

<IPython.core.display.Math object>